### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [2]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse
from sklearn.model_selection import train_test_split


In [3]:
comments = pd.read_csv("mbti_1.csv")
X_train, X_test, y_train, y_test = train_test_split(comments, comments['type'], random_state=0)
X_test.head(3)

,type,posts
4587,ISFP,"'Dear ISFJ Mother, I wish you were less of a w..."
2786,INFJ,"'To me, I think you guys may be over analyzing..."
2813,ENFP,"'NIHM While NIHM has her INTJ husband, I've go..."


In [4]:
import urllib.request
import json      


def get_prediction(x):
    posts = x
    body = {'posts': posts} 

    myurl = "http://10.0.2.15:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [12]:
#get_prediction(1,10,1,1)

In [14]:
%%time
predictions = X_test[['posts']].iloc[:500].apply(lambda x: get_prediction(x), 1)

CPU times: user 315 ms, sys: 56.6 ms, total: 372 ms
Wall time: 5.46 s


In [15]:
predictions

0      0.026174
1      0.044576
2      0.004804
3      0.001057
4      0.001229
         ...   
495    0.008572
496    0.001839
497    0.620032
498    0.001418
499    0.001099
Length: 500, dtype: float64

In [7]:
print(f1_score(y_test.iloc[:500], predictions, average='micro'))
print(f1_score(y_test.iloc[:500], predictions, average='macro'))
print(f1_score(y_test.iloc[:500], predictions, average='weighted'))
print(precision_score(y_test.iloc[:500], predictions, average='micro'))
print(precision_score(y_test.iloc[:500], predictions, average='macro'))
print(precision_score(y_test.iloc[:500], predictions, average='weighted'))

Давайте подумаем, где наше api может сломаться